<a href="https://colab.research.google.com/github/brittneyjuliet/language_music-dataset/blob/main/Music_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import numpy as np
import scipy.signal
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib import gridspec
import gspread
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import auth
auth.authenticate_user()
from google.colab import files
import gspread
from google.auth import default
creds, _ = default()
import pandas as pd

gc = gspread.authorize(creds)

# Function to Estimate Syllables
def estimate_syllables(spectrogram, threshold=0.5):
    peaks = (spectrogram > threshold).astype(int)
    syllable_count = np.sum(peaks, axis=0)
    return len(np.where(syllable_count > 0)[0])

# Create an empty DataFrame to store results
results = {"Filename": [], "Duration": [], "Syllable Count": []}

# Folder containing WAV files
folder_path = "/content/gdrive/MyDrive/language_music-dataset-main/Music_Finnish"

# Folder to save PNG files in Google Drive
png_folder_path = "/content/gdrive/MyDrive/language_music-dataset-main/PNG_files"

# Create a Google Sheet to store the information
spreadsheet = gc.create('Music_Finnish_Analysis')
worksheet = spreadsheet.sheet1
worksheet.append_row(["Filename", "Duration", "Syllable Count"])

# Iterate through the WAV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        wav_path = os.path.join(folder_path, filename)

        # Load the audio file
        y, sr = librosa.load(wav_path)

        # Calculate the spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

        # Estimate syllables
        threshold = 0.5  # Adjust as needed
        syllable_count = estimate_syllables(D, threshold)

        # Calculate duration
        duration = librosa.get_duration(y=y, sr=sr)

        # Add the results to the DataFrame
        results["Filename"].append(filename)
        results["Duration"].append(duration)
        results["Syllable Count"].append(syllable_count)

        # Save each plot as a PNG file
        fig = plt.figure(figsize=(10, 15))
        gs = gridspec.GridSpec(6, 1, height_ratios=[2, 2, 2, 2, 4, 2])

         # Pitch Analysis
        f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        plt.subplot(gs[0])
        plt.semilogy(f0, label='F0 (fundamental frequency)')
        plt.ylabel('Frequency (Hz)')
        plt.xticks([])

        # Loudness/Intensity Analysis
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        log_S = librosa.power_to_db(S, ref=np.max)
        plt.subplot(gs[1])
        librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.ylabel('Mel spectrogram')
        plt.xticks([])

        # Duration Analysis
        plt.subplot(gs[2])
        plt.text(0.5, 0.5, f'Duration: {duration:.2f} seconds', horizontalalignment='center', verticalalignment='center')
        plt.axis('off')

        # Stress Pattern (Energy Envelope)
        energy = np.sum(np.abs(librosa.stft(y)), axis=0)
        energy /= np.max(energy)  # Normalize the energy envelope
        plt.subplot(gs[3])
        plt.plot(energy, label='Energy Envelope')
        plt.ylabel('Amplitude')
        plt.xticks([])

        # Formant Frequencies
        formants = librosa.effects.preemphasis(y)
        plt.subplot(gs[4])
        librosa.display.waveshow(formants, sr=sr, alpha=0.7)
        plt.ylabel('Formants')

        plt.subplot(gs[5])
        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.ylabel('Spectrogram')

        png_folder_path = "/content/gdrive/MyDrive/language_music-dataset-main/Music_Finnish_Plots"
        os.makedirs(png_folder_path, exist_ok=True)

        # Save each plot as a PNG file
        plot_names = ["pitch_analysis, loudness_intensity, duration, stress_pattern, formant_frequencies, spectrogram"]
        for i, plot_name in enumerate(plot_names):
            plt.subplot(gs[i])
            plt.tight_layout()
            plot_filename = f"{plot_name}_{filename[:-4]}.png"
            plt.savefig(os.path.join(png_folder_path, plot_filename))


        plt.close(fig)

        # Append information to Google Sheet
        worksheet.append_row([filename, duration, syllable_count])

# Print the results DataFrame
print(pd.DataFrame(results))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
                Filename   Duration  Syllable Count
0   music_finnish_11.wav  50.645351               0
1   music_finnish_10.wav  38.545125               0
2    music_finnish_4.wav  42.240000               0
3    music_finnish_3.wav  51.029342               0
4    music_finnish_2.wav  49.920000               0
5    music_finnish_5.wav  61.909342               0
6    music_finnish_6.wav  66.645351               0
7    music_finnish_1.wav  40.320000               0
8    music_finnish_7.wav  60.416009               0
9    music_finnish_8.wav  31.021859               0
10   music_finnish_9.wav  43.003356               0
